In [78]:
import requests
import os
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from tqdm import tqdm

# **Wikipedia: S&P500 Tickers and Sector**s

In [2]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(wiki_url)
content = response.content
soup = BeautifulSoup(content, 'html.parser')

In [4]:
table = soup.find('table', class_='wikitable sortable')
rows = table.find_all('tr')[1:]

In [6]:
sectors = []
tickers = []

for row in rows:
    cells = row.find_all('td')
    sectors.append(cells[2].text.strip())
    tickers.append(cells[0].text.strip())

# **Financial Modeling Prep API: Financial Data from the last 10 years**

In [9]:
load_dotenv('.env')
api_key = os.getenv('FMP_KEY')

In [84]:
sample_ticker = 'AAPL'
sample_days = 2520
sample_endpoint = f'https://financialmodelingprep.com/api/v3/historical-price-full/{sample_ticker}?timeseries={sample_days}&apikey={api_key}'

sample_response = requests.get(sample_endpoint)
sample_data = sample_response.json()

In [85]:
columns = list(sample_data['historical'][0].keys()) + ['symbol']

In [88]:
def endpoint_string_10_years(ticker, days):
    return f'https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?timeseries={days}&apikey={api_key}'

In [90]:
sp500_df = pd.DataFrame(columns=columns)
for ticker in tqdm(tickers, desc='Collecting data'):
    endpoint = endpoint_string_10_years(ticker, 2520)
    response = requests.get(endpoint)
    data = response.json()

    if data:
        df = pd.DataFrame(sample_data['historical'])
        df['symbol'] = ticker
    else:
        print(f'No data for {ticker}.')
    sp500_df = pd.concat([sp500_df, df], ignore_index=True)

In [97]:
sp500_df.head()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,symbol
0,2023-09-27,172.62,173.040,169.05,169.53,169.53,37496091,37496091,-3.09,-1.790060,170.54,"September 27, 23",-0.017901,MMM
1,2023-09-26,174.82,175.200,171.66,171.96,171.96,64422378,61108831,-2.86,-1.640000,172.72,"September 26, 23",-0.016400,MMM
2,2023-09-25,174.20,176.970,174.15,176.08,176.08,46172740,46172700,1.88,1.080000,175.67,"September 25, 23",0.010800,MMM
3,2023-09-22,174.67,177.079,174.05,174.79,174.79,56688985,56663000,0.12,0.068701,175.47,"September 22, 23",0.000687,MMM
4,2023-09-21,174.55,176.300,173.86,173.93,173.93,62449116,63047900,-0.62,-0.355200,174.86,"September 21, 23",-0.003552,MMM


In [95]:
sp500_df.isnull().any()

date                False
open                False
high                False
low                 False
close               False
adjClose            False
volume              False
unadjustedVolume    False
change              False
changePercent       False
vwap                False
label               False
changeOverTime      False
symbol              False
dtype: bool

In [96]:
sp500_df.dtypes

date                 object
open                float64
high                float64
low                 float64
close               float64
adjClose            float64
volume               object
unadjustedVolume     object
change              float64
changePercent       float64
vwap                float64
label                object
changeOverTime      float64
symbol               object
dtype: object

In [98]:
sp500_df['date'] = pd.to_datetime(sp500_df['date'])
sp500_df['volume'] = sp500_df['volume'].astype(int)
sp500_df['unadjustedVolume'] = sp500_df['unadjustedVolume'].astype(int)

In [100]:
sp500_df.duplicated().sum()

0

In [101]:
output_file_path = './data/sp500_data.csv'
sp500_df.to_csv(output_file_path, index=False)
print(f'DataFrame has been exported to {output_file_path}')

DataFrame has been exported to ./data/sp500_data.csv
